In [21]:
%load_ext autoreload
%autoreload 2

import os
from processingmm import batch_processing
from freezingeffect import selection_of_ROIs, automatic_ROI_propagation, collect_data_propagated
from freezingeffect.helpers import load_parameters_ROIs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Process the Mueller Matrices of the folders

In [22]:
# set the parameters to run the script
data_folder_path = r'F:\ImpactHistology\Freezing_dev'
directories = [data_folder_path]
calib_directory = r'F:\polarimetry\calibration'

# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
# set the parameters to be used for the line visualisation with the parameter parameter_set
batch_processing.batch_process(directories, calib_directory, run_all = False, parameter_set = 'PD')

0it [00:00, ?it/s]


[]

## 2. Specify the folders that will be processed and clean them (remove precedent processing results)

In [23]:
path_folder_temp, path_align_folder = selection_of_ROIs.create_alignment_folder()
base_dirs = selection_of_ROIs.get_the_base_dirs(data_folder_path)
selection_of_ROIs.clean_the_alignment_folders(path_align_folder)
selection_of_ROIs.create_the_masks(data_folder_path)

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:16<00:00,  1.39s/it]


## 3. Loop over all the folders that will be processed

#### 3.1. loop over all the folders that needs to be processed

In [24]:
# tissue_types = ['WM', 'GM']
tissue_types = ['WM', 'GM']
for path_folder in base_dirs:
    for tissue_type in tissue_types:
        
        # 1. set up the parameters and load the Mueller Matrix
        WM = tissue_type == 'WM'
        
        if WM:
            new_folder_name = 'WM_1'
        else:
            new_folder_name = 'GM_1'
        propagated = False
        
        # 2. automatic selection and preparation for the propagation of the ROIs
        propagation_list = selection_of_ROIs.square_selection(path_folder_temp, path_folder, 
                                                             WM, tissue_type, path_align_folder)
        
        # 3. actually do the alignment
        propagation_list = automatic_ROI_propagation.generate_combined_mask(propagation_list, 
                                                                            path_align_folder)
        automatic_ROI_propagation.do_alignment(path_align_folder)
        output_folders = automatic_ROI_propagation.move_computed_folders(path_align_folder)
        
        # 4. and collect the data after propagation
        collect_data_propagated.collect_data_propagated(WM, path_align_folder, propagation_list, output_folders)

## 4. Compare the parameters

#### 4.1. Get the folders for the measurements of interest

In [25]:
from freezingeffect import parameter_comparaison

In [30]:
# get the parameters for the comparaison
path_folder = data_folder_path
wavelength, match_sequence, measurement, max_ = parameter_comparaison.get_parameters()


# and the folders containing the data
data_folders, base_folders, to_be_removed = parameter_comparaison.get_data_folders(path_folder, wavelength, match_sequence)

data_folders[0:5] + data_folders[-5:]

['F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_1',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_2',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_3',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_4',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_5',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S6_1\\polarimetry\\550nm\\50x50_images\\WM_1',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S6_1\\polarimetry\\550nm\\50x50_images\\WM_2',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S6_1\\polarimetry\\550nm\\50x50_images\\WM_3',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S6_1\\polarimetry\\550nm\\50x50_images\\WM_4',
 'F:\\Impa

#### 4.2. Get the data for the measurement

In [31]:
data_types, data_fixation = parameter_comparaison.get_data(data_folders)

data_fixation['GM']['F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_2']

,fname,parameter,mean,stdev,max,median,square_size
0,2023-04-06_T_FRE-FR-T0_FR_S1_1,azimuth,116.073531,22.502579,109.608939,116.073531,None
1,2023-04-06_T_FRE-FR-T0_FR_S1_1,depolarization,0.804452,0.038905,0.849265,0.807704,None
2,2023-04-06_T_FRE-FR-T0_FR_S1_1,diattenuation,0.028094,0.009425,0.025253,0.027571,None
3,2023-04-06_T_FRE-FR-T0_FR_S1_1,retardance,11.985394,4.766775,11.564246,11.881654,None
4,2023-04-11_T_FRE-FR-T1_FR_S1_1,azimuth,9.736733,33.239416,10.055866,9.736733,None
5,2023-04-11_T_FRE-FR-T1_FR_S1_1,depolarization,0.740573,0.019267,0.744118,0.741195,None
6,2023-04-11_T_FRE-FR-T1_FR_S1_1,diattenuation,0.038327,0.009681,0.040404,0.038069,None
7,2023-04-11_T_FRE-FR-T1_FR_S1_1,retardance,5.655211,2.769873,6.033520,5.323948,None


In [32]:
# remove the ROI moving from white matter to grey matter and vice versa
for data_type, dfed in data_fixation.items():
    for folder, df in dfed.items():
        to_remove_dfed = to_be_removed[folder.split('\\polarimetry')[0]]
        for val in to_remove_dfed:
            for fol in val:
                if fol[0] == folder.split('\\')[-1].split('_align')[0]:
                    df = df.apply(parameter_comparaison.find_match_seq, match_seq = fol[1], axis = 1)
                    data_fixation[data_type][folder] = df

In [33]:
data_fixation['GM']['F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_2']

,fname,parameter,mean,stdev,max,median,square_size
0,2023-04-06_T_FRE-FR-T0_FR_S1_1,azimuth,116.073531,22.502579,109.608939,116.073531,None
1,2023-04-06_T_FRE-FR-T0_FR_S1_1,depolarization,0.804452,0.038905,0.849265,0.807704,None
2,2023-04-06_T_FRE-FR-T0_FR_S1_1,diattenuation,0.028094,0.009425,0.025253,0.027571,None
3,2023-04-06_T_FRE-FR-T0_FR_S1_1,retardance,11.985394,4.766775,11.564246,11.881654,None
4,2023-04-11_T_FRE-FR-T1_FR_S1_1,azimuth,9.736733,33.239416,10.055866,9.736733,None
5,2023-04-11_T_FRE-FR-T1_FR_S1_1,depolarization,0.740573,0.019267,0.744118,0.741195,None
6,2023-04-11_T_FRE-FR-T1_FR_S1_1,diattenuation,0.038327,0.009681,0.040404,0.038069,None
7,2023-04-11_T_FRE-FR-T1_FR_S1_1,retardance,5.655211,2.769873,6.033520,5.323948,None


In [44]:
#### Set up the parameters to be studied
parameters = ['retardance', 'depolarization', 'diattenuation', 'azimuth']
metrics = ['median', 'mean', 'max']
times = ['T0', 'T1']
    
# create the directories that will be used to store the output of the analysis
folder = parameter_comparaison.create_output_directories(measurement, data_types, parameters = parameters)

In [45]:
### Get the data and save it into xlsx file
to_combine = parameter_comparaison.get_and_organize_data(data_fixation, metrics, data_types, times, folder, max_,
                          parameters = ['retardance', 'depolarization', 'diattenuation', 'azimuth'])

to_combine[('retardance', 'WM', 'median')]

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,11.563831,11.297006,16.808342,12.562762,12.564425,25.697801,21.680195,16.959161,19.557084,13.629349,...,15.468305,12.667431,20.422814,20.828679,36.811760,23.025728,30.592759,15.926166,11.925587,17.377520
1,17.891148,NaN,NaN,11.784891,12.562555,22.032371,20.947990,19.192360,26.600240,14.363512,...,12.229231,14.293533,17.775943,15.055323,15.570766,15.049274,7.735759,14.731747,12.912673,16.914606


In [54]:
for idx in range(1,3):
    print(idx)

1
2


In [55]:
to_combine_cv = {}
for idx, val in to_combine.items():
    if idx[0] == 'azimuth':
        to_combine_cv[idx] = parameter_comparaison.combine_data_cv(val, len(times), azimuth = True)
    else:
        to_combine_cv[idx] = parameter_comparaison.combine_data_cv(val, len(times))
        
to_combine_cv[('retardance', 'WM', 'median')]

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000
1,1.547164,NaN,NaN,0.938081,0.999851,0.857364,0.966227,1.131681,1.360133,1.053866,...,0.790599,1.128369,0.870396,0.722817,0.422983,0.653585,0.252862,0.925003,1.08277,0.973361


#### 4.3. Statistical analysis

In [56]:
#### Get the data to be used for the comparisons
# recombine the data in a different format
data_all_recombined = parameter_comparaison.recombine_data_tests(to_combine, times)
data_all_recombined_cv = parameter_comparaison.recombine_data_tests(to_combine_cv, times)

In [58]:
#### Comparaison of the means and the means of the fold change
# perform the statistical test for both fold change and mean values
paired_test = parameter_comparaison.t_test(data_all_recombined, times, 'median')# times[1:]])
paired_test_cv = parameter_comparaison.t_test(data_all_recombined_cv, times, 'median')# times[1:]])
df_grouped = parameter_comparaison.create_test_df(paired_test, parameter = 'median')
df_grouped_cv = parameter_comparaison.create_test_df(paired_test_cv, parameter = 'median')
df_grouped_cv

Z       p_value   n_1   n_2  before/GM  \
parameter      data_type time                                               
azimuth        GM        T0    870.0  1.566732e-12  29.0  30.0  30.148747   
                         T1    420.5  1.000000e+00  29.0  29.0  30.148747   
               WM        T0    750.0  4.655375e-12  25.0  30.0  19.643097   
                         T1    312.5  1.000000e+00  25.0  25.0  19.643097   
depolarization GM        T0    360.0  2.255251e-01  29.0  30.0   1.002618   
                         T1    420.5  1.000000e+00  29.0  29.0   1.002618   
               WM        T0    450.0  1.688976e-01  25.0  30.0   1.001088   
                         T1    312.5  1.000000e+00  25.0  25.0   1.001088   
retardance     GM        T0    330.0  8.912959e-02  29.0  30.0   0.821817   
                         T1    420.5  1.000000e+00  29.0  29.0   0.821817   
               WM        T0    390.0  7.888807e-01  25.0  30.0   1.028790   
                         T1    312.5  1.000000e+00  25.0  25.0   1.028790   

                               before/GM std   after/WM  after/WM std  
parameter      data_type time                                          
azimuth        GM        T0        19.924745   0.000000      0.000000  
                         T1        19.924745  30.148747     19.924745  
               WM        T0        16.598904   0.000000      0.000000  
                         T1        16.598904  19.643097     16.598904  
depolarization GM        T0         0.049795   1.000000      0.000000  
                         T1         0.049795   1.002618      0.049795  
               WM        T0         0.020772   1.000000      0.000000  
                         T1         0.020772   1.001088      0.020772  
retardance     GM        T0         0.365678   1.000000      0.000000  
                         T1         0.365678   0.821817      0.365678  
               WM        T0         0.316989   1.000000      0.000000  
                         T1         0.316989   1.028790      0.316989

In [59]:
# finally, save the results
path_statistical_test = os.path.join('results', measurement)
df_grouped.to_excel(os.path.join(path_statistical_test, 'statistical_test.xlsx'))
df_grouped_cv.to_excel(os.path.join(path_statistical_test, 'statistical_test_cv.xlsx'))

#### 4.4. Combine the results in a single excel file

In [60]:
# this cell allows to combine the data obtained for both grey and white matter in a table that can then be used
# to plot the results using Graphpad Prism
retardance = {}
depolarization = {}
for key, param in to_combine.items():
    if key[0] == 'retardance' and key[2] == 'median':
        retardance[key[1]] = param
    elif key[0] == 'depolarization' and key[2] == 'median':
        depolarization[key[1]] = param
        
retardance_cv = {}
depolarization_cv = {}
azimuth_cv = {}
for key, param in to_combine_cv.items():
    if key[0] == 'retardance' and key[2] == 'median':
        retardance_cv[key[1]] = param
    elif key[0] == 'depolarization' and key[2] == 'median':
        depolarization_cv[key[1]] = param
    else:
        if key[2] == 'mean':
            azimuth_cv[key[1]] = param
        
retardance = parameter_comparaison.create_df_prism(retardance, max_)
depolarization = parameter_comparaison.create_df_prism(depolarization, max_)
    
retardance_cv = parameter_comparaison.create_df_prism(retardance_cv, max_)
depolarization_cv = parameter_comparaison.create_df_prism(depolarization_cv, max_)
azimuth_cv = parameter_comparaison.create_df_prism(azimuth_cv, max_)

retardance_cv

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,1.0,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000
1,NaN,0.448081,0.47506,1.013051,1.175285,0.587094,1.165967,0.482251,0.713287,0.629641,...,0.790599,1.128369,0.870396,0.722817,0.422983,0.653585,0.252862,0.925003,1.08277,0.973361


In [61]:
# finally save all the tables
path_data_prism = os.path.join('results', measurement)
retardance.to_excel(os.path.join(path_data_prism, "retardance_prism.xlsx"))
depolarization.to_excel(os.path.join(path_data_prism, "depolarization_prism.xlsx"))
retardance_cv.to_excel(os.path.join(path_data_prism, "retardance_prism_cv.xlsx"))
depolarization_cv.to_excel(os.path.join(path_data_prism, "depolarization_prism_cv.xlsx"))
azimuth_cv.to_excel(os.path.join(path_data_prism, "azimuth_prism_cv.xlsx"))